# IEEE 2019 - XGBoost

In [0]:
!pip install knockknock

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, warnings, random, datetime, time
import pickle
from knockknock import telegram_sender

from sklearn import metrics
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import math
warnings.filterwarnings('ignore')

# Pandas display format
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

In [0]:
# Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# Memory Reducer
def memory_usage_mb(df, *args, **kwargs):
    """Dataframe memory usage in MB. """
    return df.memory_usage(*args, **kwargs).sum() / 1024**2

def reduce_mem_usage(df, deep=True, verbose=False, categories=False):
    # All types that we want to change for "lighter" ones.
    # int8 and float16 are not include because we cannot reduce
    # those data types.
    # float32 is not include because float16 has too low precision.
    numeric2reduce = ["int16", "int32", "int64", "float64"]
    start_mem = 0
    start_mem = memory_usage_mb(df, deep=deep)

    for col, col_type in df.dtypes.iteritems():
        best_type = None
        if categories:
            if col_type == "object":
                df[col] = df[col].astype("category")
                best_type = "category"
        elif col_type in numeric2reduce:
            downcast = "integer" if "int" in str(col_type) else "float"
            df[col] = pd.to_numeric(df[col], downcast=downcast)
            best_type = df[col].dtype.name
        # Log the conversion performed.
        if verbose and best_type is not None and best_type != str(col_type):
            print(f"Column '{col}' converted from {col_type} to {best_type}")

    end_mem = memory_usage_mb(df, deep=deep)
    diff_mem = start_mem - end_mem
    percent_mem = 100 * diff_mem / start_mem
    print(f"Memory usage decreased from"
          f" {start_mem:.2f}MB to {end_mem:.2f}MB"
          f" ({diff_mem:.2f}MB, {percent_mem:.2f}% reduction)")
    
    return df

In [0]:
# Custom Focal Loss

def robust_pow(num_base, num_pow):
    # numpy does not permit negative numbers to fractional power
    # use this to perform the power algorithmic

    return np.sign(num_base) * (np.abs(num_base)) ** (num_pow)

def focal_binary_object(y_pred, dtrain):
    gamma_indct = 2.0
    # retrieve data from dtrain matrix
    label = dtrain.get_label()
    # compute the prediction with sigmoid
    sigmoid_pred = 1.0 / (1.0 + np.exp(-y_pred))
    # gradient
    # complex gradient with different parts
    g1 = sigmoid_pred * (1 - sigmoid_pred)
    g2 = label + ((-1) ** label) * sigmoid_pred
    g3 = sigmoid_pred + label - 1
    g4 = 1 - label - ((-1) ** label) * sigmoid_pred
    g5 = label + ((-1) ** label) * sigmoid_pred
    # combine the gradient
    grad = gamma_indct * g3 * robust_pow(g2, gamma_indct) * np.log(g4 + 1e-9) + \
           ((-1) ** label) * robust_pow(g5, (gamma_indct + 1))
    # combine the gradient parts to get hessian components
    hess_1 = robust_pow(g2, gamma_indct) + \
             gamma_indct * ((-1) ** label) * g3 * robust_pow(g2, (gamma_indct - 1))
    hess_2 = ((-1) ** label) * g3 * robust_pow(g2, gamma_indct) / g4
    # get the final 2nd order derivative
    hess = ((hess_1 * np.log(g4 + 1e-9) - hess_2) * gamma_indct +
            (gamma_indct + 1) * robust_pow(g5, gamma_indct)) * g1

    return grad, hess

In [0]:
# Vars

SEED = 42
seed_everything(SEED)
LOCAL_TEST = False
TARGET = 'isFraud'
PREDICT = True

In [0]:
# DATA LOAD

print('Load Data')
train_df = pd.read_pickle('../content/gdrive/My Drive/IEEE fraud Kaggle 2019/train_full_feat.pkl')

if LOCAL_TEST:
    test_df = train_df[train_df['DT_M']==train_df['DT_M'].max()].reset_index(drop=True)
    train_df = train_df[train_df['DT_M']<(train_df['DT_M'].max()-1)].reset_index(drop=True)    
else:
    test_df = pd.read_pickle('../content/gdrive/My Drive/IEEE fraud Kaggle 2019/test_full_feat.pkl')
    test_df['isFraud'] = 0

print('Shape control:\nTrain:', train_df.shape, '\nTest:',test_df.shape)

Load Data
Shape control:
Train: (590540, 447) 
Test: (506691, 446)


In [0]:
train_df.head()

,TransactionID,isFraud,Amt_timeslowest_first_UserID_proxy,Amt_timeslowest_first_group,Amtisfirst_UserID_proxy,Amtisfirst_group,C1,C10,C11,C12,C13,C14,C2,C4,C5,C6,C7,C8,C9,D10,D10_DT_D_min_max,D10_DT_D_std_score,D10_DT_M_min_max,D10_DT_M_std_score,D10_DT_W_min_max,D10_DT_W_std_score,D11,D11_DT_D_min_max,D11_DT_D_std_score,D11_DT_M_min_max,D11_DT_M_std_score,D11_DT_W_min_max,D11_DT_W_std_score,D11__DeviceInfo,D12,D12_DT_D_min_max,D12_DT_D_std_score,D12_DT_M_min_max,D12_DT_M_std_score,D12_DT_W_min_max,D12_DT_W_std_score,D13,D13_DT_D_min_max,D13_DT_D_std_score,D13_DT_M_min_max,D13_DT_M_std_score,D13_DT_W_min_max,D13_DT_W_std_score,D14,D14_DT_D_min_max,D14_DT_D_std_score,D14_DT_M_min_max,D14_DT_M_std_score,D14_DT_W_min_max,D14_DT_W_std_score,D15,D15_DT_D_min_max,D15_DT_D_std_score,D15_DT_M_min_max,D15_DT_M_std_score,D15_DT_W_min_max,D15_DT_W_std_score,D2,D2_scaled,D3,D3_DT_D_min_max,D3_DT_D_std_score,D3_DT_M_min_max,D3_DT_M_std_score,D3_DT_W_min_max,D3_DT_W_std_score,D4,D4_DT_D_min_max,D4_DT_D_std_score,D4_DT_M_min_max,D4_DT_M_std_score,D4_DT_W_min_max,D4_DT_W_std_score,D5,D5_DT_D_min_max,D5_DT_D_std_score,D5_DT_M_min_max,D5_DT_M_std_score,D5_DT_W_min_max,D5_DT_W_std_score,D6,D6_DT_D_min_max,D6_DT_D_std_score,D6_DT_M_min_max,D6_DT_M_std_score,D6_DT_W_min_max,D6_DT_W_std_score,D7_DT_D_min_max,D7_DT_D_std_score,D7_DT_M_min_max,D7_DT_M_std_score,D7_DT_W_min_max,D7_DT_W_std_score,D8,D8_D9_decimal_dist,D8_DT_D_min_max,D8_DT_D_std_score,D8_DT_M_min_max,D8_DT_M_std_score,D8_DT_W_min_max,D8_DT_W_std_score,D8__D9,D8_not_same_day,D9,D9_not_na,DT_M,DeviceInfo,DeviceInfo__P_emaildomain,DeviceInfo_device,DeviceInfo_version,DeviceType,M2,M2__M3,M3,M4,M5,M6,M7,M8,M9,P_emaildomain,P_emaildomain__C2,ProductCD,R_emaildomain,TransactionAmt,TransactionAmt_DT_D_min_max,TransactionAmt_DT_D_std_score,TransactionAmt_DT_M_min_max,TransactionAmt_DT_M_std_score,TransactionAmt_DT_W_min_max,TransactionAmt_DT_W_std_score,TransactionAmt_check,V_12_34_0,V_12_34_1,V_12_34_10,V_12_34_11,V_12_34_2,V_12_34_3,V_12_34_4,V_12_34_5,V_12_34_6,V_12_34_7,V_12_34_8,V_12_34_9,V_138_166_0,V_167_216_0,V_167_216_1,V_167_216_2,V_1_11_0,V_1_11_1,V_1_11_2,V_1_11_3,V_1_11_4,V_1_11_5,V_1_11_6,V_217_278_0,V_217_278_1,V_217_278_2,V_279_321_0,V_279_321_1,V_279_321_2,V_279_321_3,V_322_339_0,V_322_339_1,V_35_52_0,V_35_52_1,V_35_52_2,V_35_52_3,V_35_52_4,V_35_52_5,V_35_52_6,V_35_52_7,V_35_52_8,V_53_74_0,V_53_74_1,V_53_74_10,V_53_74_11,V_53_74_2,V_53_74_3,V_53_74_4,V_53_74_5,V_53_74_6,V_53_74_7,V_53_74_8,V_53_74_9,V_75_94_0,V_75_94_1,V_75_94_10,V_75_94_2,V_75_94_3,V_75_94_4,V_75_94_5,V_75_94_6,V_75_94_7,V_75_94_8,V_75_94_9,V_95_137_0,V_95_137_1,V_95_137_2,addr1,addr1__card1,addr2,amplitude_Amt_UserID,amplitude_Time_UserID,amplitude_dist1_UserID,amplitude_dist2_UserID,card1,card1_TransactionAmt_mean,card1__card5,card2,card2_TransactionAmt_mean,card2__dist1,card2__id_20,card3,card3_TransactionAmt_mean,card4,card5,card5_TransactionAmt_mean,card5__P_emaildomain,card6,cardID_v2_amplitude_Amt_group,cardID_v2_amplitude_Time_group,cardID_v2_amplitude_dist1_group,cardID_v2_amplitude_dist2_group,cardID_v2_count_group,cardID_v2_first_Amt_group,cardID_v2_max_D3_group,cardID_v2_mean_Amt_group,cardID_v2_mean_C10_group,cardID_v2_mean_C11_group,cardID_v2_mean_C12_group,cardID_v2_mean_C13_group,cardID_v2_mean_C14_group,cardID_v2_mean_C1_group,cardID_v2_mean_C2_group,cardID_v2_mean_C3_group,cardID_v2_mean_C4_group,cardID_v2_mean_C5_group,cardID_v2_mean_C6_group,cardID_v2_mean_C7_group,cardID_v2_mean_C8_group,cardID_v2_mean_C9_group,cardID_v2_mean_D3_group,cardID_v2_median_Amt_group,cardID_v2_skew_Amt_group,cardID_v2_unique_Amt_group,cardID_v2_unique_Device,cardID_v2_unique_IP,cardID_v2_unique_Pemail_group,cardID_v2_unique_Remail_group,cardID_v2_unique_adr1_group,cardID_v2_unique_adr2_group,cardID_v2_unique_id_30,cardID_v2_unique_id_31,cardID_v2_unique_id_33,cardID_v2_var_Amt_group,cardID_v2_var_C10_group,cardID_v2_var_C11_group,cardID_v2_var_C12_group,cardID_v2_var_C13_group,cardID_v2_var_C14_group,cardID_v2_

In [0]:
test_df.head()

,TransactionID,isFraud,Amt_timeslowest_first_UserID_proxy,Amt_timeslowest_first_group,Amtisfirst_UserID_proxy,Amtisfirst_group,C1,C10,C11,C12,C13,C14,C2,C4,C5,C6,C7,C8,C9,D10,D10_DT_D_min_max,D10_DT_D_std_score,D10_DT_M_min_max,D10_DT_M_std_score,D10_DT_W_min_max,D10_DT_W_std_score,D11,D11_DT_D_min_max,D11_DT_D_std_score,D11_DT_M_min_max,D11_DT_M_std_score,D11_DT_W_min_max,D11_DT_W_std_score,D11__DeviceInfo,D12,D12_DT_D_min_max,D12_DT_D_std_score,D12_DT_M_min_max,D12_DT_M_std_score,D12_DT_W_min_max,D12_DT_W_std_score,D13,D13_DT_D_min_max,D13_DT_D_std_score,D13_DT_M_min_max,D13_DT_M_std_score,D13_DT_W_min_max,D13_DT_W_std_score,D14,D14_DT_D_min_max,D14_DT_D_std_score,D14_DT_M_min_max,D14_DT_M_std_score,D14_DT_W_min_max,D14_DT_W_std_score,D15,D15_DT_D_min_max,D15_DT_D_std_score,D15_DT_M_min_max,D15_DT_M_std_score,D15_DT_W_min_max,D15_DT_W_std_score,D2,D2_scaled,D3,D3_DT_D_min_max,D3_DT_D_std_score,D3_DT_M_min_max,D3_DT_M_std_score,D3_DT_W_min_max,D3_DT_W_std_score,D4,D4_DT_D_min_max,D4_DT_D_std_score,D4_DT_M_min_max,D4_DT_M_std_score,D4_DT_W_min_max,D4_DT_W_std_score,D5,D5_DT_D_min_max,D5_DT_D_std_score,D5_DT_M_min_max,D5_DT_M_std_score,D5_DT_W_min_max,D5_DT_W_std_score,D6,D6_DT_D_min_max,D6_DT_D_std_score,D6_DT_M_min_max,D6_DT_M_std_score,D6_DT_W_min_max,D6_DT_W_std_score,D7_DT_D_min_max,D7_DT_D_std_score,D7_DT_M_min_max,D7_DT_M_std_score,D7_DT_W_min_max,D7_DT_W_std_score,D8,D8_D9_decimal_dist,D8_DT_D_min_max,D8_DT_D_std_score,D8_DT_M_min_max,D8_DT_M_std_score,D8_DT_W_min_max,D8_DT_W_std_score,D8__D9,D8_not_same_day,D9,D9_not_na,DeviceInfo,DeviceInfo__P_emaildomain,DeviceInfo_device,DeviceInfo_version,DeviceType,M2,M2__M3,M3,M4,M5,M6,M7,M8,M9,P_emaildomain,P_emaildomain__C2,ProductCD,R_emaildomain,TransactionAmt,TransactionAmt_DT_D_min_max,TransactionAmt_DT_D_std_score,TransactionAmt_DT_M_min_max,TransactionAmt_DT_M_std_score,TransactionAmt_DT_W_min_max,TransactionAmt_DT_W_std_score,TransactionAmt_check,V_12_34_0,V_12_34_1,V_12_34_10,V_12_34_11,V_12_34_2,V_12_34_3,V_12_34_4,V_12_34_5,V_12_34_6,V_12_34_7,V_12_34_8,V_12_34_9,V_138_166_0,V_167_216_0,V_167_216_1,V_167_216_2,V_1_11_0,V_1_11_1,V_1_11_2,V_1_11_3,V_1_11_4,V_1_11_5,V_1_11_6,V_217_278_0,V_217_278_1,V_217_278_2,V_279_321_0,V_279_321_1,V_279_321_2,V_279_321_3,V_322_339_0,V_322_339_1,V_35_52_0,V_35_52_1,V_35_52_2,V_35_52_3,V_35_52_4,V_35_52_5,V_35_52_6,V_35_52_7,V_35_52_8,V_53_74_0,V_53_74_1,V_53_74_10,V_53_74_11,V_53_74_2,V_53_74_3,V_53_74_4,V_53_74_5,V_53_74_6,V_53_74_7,V_53_74_8,V_53_74_9,V_75_94_0,V_75_94_1,V_75_94_10,V_75_94_2,V_75_94_3,V_75_94_4,V_75_94_5,V_75_94_6,V_75_94_7,V_75_94_8,V_75_94_9,V_95_137_0,V_95_137_1,V_95_137_2,addr1,addr1__card1,addr2,amplitude_Amt_UserID,amplitude_Time_UserID,amplitude_dist1_UserID,amplitude_dist2_UserID,card1,card1_TransactionAmt_mean,card1__card5,card2,card2_TransactionAmt_mean,card2__dist1,card2__id_20,card3,card3_TransactionAmt_mean,card4,card5,card5_TransactionAmt_mean,card5__P_emaildomain,card6,cardID_v2_amplitude_Amt_group,cardID_v2_amplitude_Time_group,cardID_v2_amplitude_dist1_group,cardID_v2_amplitude_dist2_group,cardID_v2_count_group,cardID_v2_first_Amt_group,cardID_v2_max_D3_group,cardID_v2_mean_Amt_group,cardID_v2_mean_C10_group,cardID_v2_mean_C11_group,cardID_v2_mean_C12_group,cardID_v2_mean_C13_group,cardID_v2_mean_C14_group,cardID_v2_mean_C1_group,cardID_v2_mean_C2_group,cardID_v2_mean_C3_group,cardID_v2_mean_C4_group,cardID_v2_mean_C5_group,cardID_v2_mean_C6_group,cardID_v2_mean_C7_group,cardID_v2_mean_C8_group,cardID_v2_mean_C9_group,cardID_v2_mean_D3_group,cardID_v2_median_Amt_group,cardID_v2_skew_Amt_group,cardID_v2_unique_Amt_group,cardID_v2_unique_Device,cardID_v2_unique_IP,cardID_v2_unique_Pemail_group,cardID_v2_unique_Remail_group,cardID_v2_unique_adr1_group,cardID_v2_unique_adr2_group,cardID_v2_unique_id_30,cardID_v2_unique_id_31,cardID_v2_unique_id_33,cardID_v2_var_Amt_group,cardID_v2_var_C10_group,cardID_v2_var_C11_group,cardID_v2_var_C12_group,cardID_v2_var_C13_group,cardID_v2_var_C14_group,cardID_v2_var_C

In [0]:
# Model
import xgboost as xgb

def make_predictions(tr_df, tt_df, features_columns, target, xgb_params, logfile, custom_object, NFOLDS=2):
  
  folds = GroupKFold(n_splits=NFOLDS)

  X,y = tr_df[features_columns], tr_df[target]    
  P,P_y = tt_df[features_columns], tt_df[target]  
  split_groups = tr_df['DT_M']

  tt_df = tt_df[['TransactionID',target]]    
  predictions = np.zeros(len(tt_df))
  oof = np.zeros(len(tr_df))
  oof_preds_df = pd.DataFrame(columns=['TransactionID','oof_preds'])
  feature_importance = pd.DataFrame(columns=['Feature','Gain Value','Fold'])
  mean_tr_auc = 0
  mean_oof_auc = 0

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)):
    #if fold_ in [4]: # Use this+indent to check specific folds
    # ------
    print('Fold:',fold_)
    print('Fold:',fold_, file=logfile)

    tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
    vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]

    print('Tr len:',len(tr_x),'- Val len:',len(vl_x))

    dtrain = xgb.DMatrix(tr_x, label=tr_y)
    dval = xgb.DMatrix(vl_x, label=vl_y)

    watchlist = [(dtrain, 'train'),(dval, 'val')]

    xgbclf = xgb.train(xgb_params,
                       dtrain,
                       num_boost_round=1800,
                       evals=watchlist,
                       obj=custom_object, #comment this if not using custom obj
                       verbose_eval=300)

    dtest = xgb.DMatrix(P)
    pp_p = xgbclf.predict(dtest)
    predictions += pp_p/NFOLDS

    oof_preds = xgbclf.predict(dval)
    tr_preds_f = xgbclf.predict(dtrain)

    oof_preds_f = pd.DataFrame()
    oof_preds_f['oof_preds'] = oof_preds
    oof_preds_f['TransactionID'] = tr_df.iloc[val_idx,:][['TransactionID']].reset_index(drop=True)
    oof_preds_f = oof_preds_f[['TransactionID','oof_preds']]

    oof_preds_df = oof_preds_df.append(oof_preds_f)

    oof[val_idx] = oof_preds


    feature_imp = pd.DataFrame(list(xgbclf.get_score(importance_type='gain').items()), columns=['Feature','Gain Value'])
    feature_imp['Fold'] = fold_
    feature_importance = feature_importance.append(feature_imp)

    fold_auc = metrics.roc_auc_score(vl_y, oof_preds)
    tr_auc_f = metrics.roc_auc_score(tr_y, tr_preds_f)
    mean_tr_auc += tr_auc_f/NFOLDS
    mean_oof_auc += fold_auc/NFOLDS

    print('Train Fold AUC:', tr_auc_f, '- Val Fold AUC:', fold_auc, file=logfile)
    print('Train Fold AUC:', tr_auc_f, '- Val Fold AUC:', fold_auc)

    del tr_x, tr_y, vl_x, vl_y, oof_preds_f, feature_imp
    gc.collect()
    # -------

  tt_df['prediction'] = predictions
  
  dtrain_tot = xgb.DMatrix(X)
  tr_preds = xgbclf.predict(dtrain_tot)
  tr_score = metrics.roc_auc_score(y, tr_preds)
  oof_score = metrics.roc_auc_score(y, oof)

  if not LOCAL_TEST:
    oof_preds_df.to_csv('../content/gdrive/My Drive/IEEE fraud Kaggle 2019/xgb_output/train_f_xgb_focal_all_feat_'+str(round(mean_oof_auc,4))+'.csv',index=False)
    feature_importance.to_csv('../content/gdrive/My Drive/IEEE fraud Kaggle 2019/xgb_output/feature_imp_focal_all_feat_'+str(round(mean_oof_auc,4))+'.csv',index=False)

  print('-'*30)
  print('-'*30, file=logfile)
  
  print('Train MEAN fold AUC:', mean_tr_auc, '- Train AUC:', tr_score, file=logfile)
  print('Train MEAN fold AUC:', mean_tr_auc, '- Train AUC:', tr_score)
  
  print('OOF MEAN AUC:', mean_oof_auc, '- OOF AUC:', oof_score, file=logfile)
  print('OOF MEAN AUC:', mean_oof_auc, '- OOF AUC:', oof_score)

  if LOCAL_TEST:
    print('Holdout AUC:', metrics.roc_auc_score(tt_df[TARGET], tt_df['prediction']))

  return tt_df, mean_oof_auc

In [0]:
# Final features list
#features_columns = [col for col in train_df.columns if col not in ['TransactionID','isFraud','DT_M']]
features_columns = ['Amt_timeslowest_first_UserID_proxy',
 'Amt_timeslowest_first_group',
 'Amtisfirst_UserID_proxy',
 'Amtisfirst_group',
 'C1',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'D10_DT_D_min_max',
 'D10_DT_D_std_score',
 'D10_DT_M_min_max',
 'D10_DT_M_std_score',
 'D10_DT_W_min_max',
 'D10_DT_W_std_score',
 'D11',
 'D11_DT_D_min_max',
 'D11_DT_D_std_score',
 'D11_DT_M_min_max',
 'D11_DT_M_std_score',
 'D11_DT_W_min_max',
 'D11_DT_W_std_score',
 'D11__DeviceInfo',
 'D12',
 'D12_DT_D_min_max',
 'D12_DT_D_std_score',
 'D12_DT_M_min_max',
 'D12_DT_M_std_score',
 'D12_DT_W_min_max',
 'D12_DT_W_std_score',
 'D13',
 'D13_DT_D_min_max',
 'D13_DT_D_std_score',
 'D13_DT_M_min_max',
 'D13_DT_M_std_score',
 'D13_DT_W_min_max',
 'D13_DT_W_std_score',
 'D14',
 'D14_DT_D_min_max',
 'D14_DT_D_std_score',
 'D14_DT_M_min_max',
 'D14_DT_M_std_score',
 'D14_DT_W_min_max',
 'D14_DT_W_std_score',
 'D15',
 'D15_DT_D_min_max',
 'D15_DT_D_std_score',
 'D15_DT_M_min_max',
 'D15_DT_M_std_score',
 'D15_DT_W_min_max',
 'D15_DT_W_std_score',
 'D2',
 'D2_scaled',
 'D3',
 'D3_DT_D_min_max',
 'D3_DT_D_std_score',
 'D3_DT_M_min_max',
 'D3_DT_M_std_score',
 'D3_DT_W_min_max',
 'D3_DT_W_std_score',
 'D4',
 'D4_DT_D_min_max',
 'D4_DT_D_std_score',
 'D4_DT_M_min_max',
 'D4_DT_M_std_score',
 'D4_DT_W_min_max',
 'D4_DT_W_std_score',
 'D5',
 'D5_DT_D_min_max',
 'D5_DT_D_std_score',
 'D5_DT_M_min_max',
 'D5_DT_M_std_score',
 'D5_DT_W_min_max',
 'D5_DT_W_std_score',
 'D6',
 'D6_DT_D_min_max',
 'D6_DT_D_std_score',
 'D6_DT_M_min_max',
 'D6_DT_M_std_score',
 'D6_DT_W_min_max',
 'D6_DT_W_std_score',
 'D7_DT_D_min_max',
 'D7_DT_D_std_score',
 'D7_DT_M_min_max',
 'D7_DT_M_std_score',
 'D7_DT_W_min_max',
 'D7_DT_W_std_score',
 'D8',
 'D8_D9_decimal_dist',
 'D8_DT_D_min_max',
 'D8_DT_D_std_score',
 'D8_DT_M_min_max',
 'D8_DT_M_std_score',
 'D8_DT_W_min_max',
 'D8_DT_W_std_score',
 'D8__D9',
 'D8_not_same_day',
 'D9',
 'D9_not_na',
 'DeviceInfo',
 'DeviceInfo__P_emaildomain',
 'DeviceInfo_device',
 'DeviceInfo_version',
 'DeviceType',
 'M2',
 'M2__M3',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'P_emaildomain',
 'P_emaildomain__C2',
 'ProductCD',
 'R_emaildomain',
 'TransactionAmt',
 'TransactionAmt_DT_D_min_max',
 'TransactionAmt_DT_D_std_score',
 'TransactionAmt_DT_M_min_max',
 'TransactionAmt_DT_M_std_score',
 'TransactionAmt_DT_W_min_max',
 'TransactionAmt_DT_W_std_score',
 'TransactionAmt_check',
 'V_12_34_0',
 'V_12_34_1',
 'V_12_34_10',
 'V_12_34_11',
 'V_12_34_2',
 'V_12_34_3',
 'V_12_34_4',
 'V_12_34_5',
 'V_12_34_6',
 'V_12_34_7',
 'V_12_34_8',
 'V_12_34_9',
 'V_138_166_0',
 'V_167_216_0',
 'V_167_216_1',
 'V_167_216_2',
 'V_1_11_0',
 'V_1_11_1',
 'V_1_11_2',
 'V_1_11_3',
 'V_1_11_4',
 'V_1_11_5',
 'V_1_11_6',
 'V_217_278_0',
 'V_217_278_1',
 'V_217_278_2',
 'V_279_321_0',
 'V_279_321_1',
 'V_279_321_2',
 'V_279_321_3',
 'V_322_339_0',
 'V_322_339_1',
 'V_35_52_0',
 'V_35_52_1',
 'V_35_52_2',
 'V_35_52_3',
 'V_35_52_4',
 'V_35_52_5',
 'V_35_52_6',
 'V_35_52_7',
 'V_35_52_8',
 'V_53_74_0',
 'V_53_74_1',
 'V_53_74_10',
 'V_53_74_11',
 'V_53_74_2',
 'V_53_74_3',
 'V_53_74_4',
 'V_53_74_5',
 'V_53_74_6',
 'V_53_74_7',
 'V_53_74_8',
 'V_53_74_9',
 'V_75_94_0',
 'V_75_94_1',
 'V_75_94_10',
 'V_75_94_2',
 'V_75_94_3',
 'V_75_94_4',
 'V_75_94_5',
 'V_75_94_6',
 'V_75_94_7',
 'V_75_94_8',
 'V_75_94_9',
 'V_95_137_0',
 'V_95_137_1',
 'V_95_137_2',
 'addr1',
 'addr1__card1',
 'addr2',
 'card1',
 'card1_TransactionAmt_mean',
 'card1__card5',
 'card2',
 'card2_TransactionAmt_mean',
 'card2__dist1',
 'card2__id_20',
 'card3',
 'card3_TransactionAmt_mean',
 'card4',
 'card5',
 'card5_TransactionAmt_mean',
 'card5__P_emaildomain',
 'card6',
 'cardID_v2_count_group',
 'cardID_v2_median_Amt_group',
 'cardID_v2_skew_Amt_group',
 'cardID_v2_unique_Amt_group',
 'cardID_v2_unique_Device',
 'cardID_v2_unique_IP',
 'cardID_v2_unique_Pemail_group',
 'cardID_v2_unique_Remail_group',
 'cardID_v2_unique_adr1_group',
 'cardID_v2_unique_adr2_group',
 'cardID_v2_var_Amt_group',
 'cardID_v2_var_Time_group',
 'count_UserID',
 'dist1',
 'first_Amt_UserID',
 'id_01',
 'id_02',
 'id_02__D8',
 'id_02__id_20',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_30_device',
 'id_30_version',
 'id_31',
 'id_31_device',
 'id_32',
 'id_33',
 'id_33_0',
 'id_33_1',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'is_holiday',
 'median_Amt_UserID',
 'product_type',
 'product_type_DT_D',
 'product_type_DT_M',
 'product_type_DT_W',
 'skew_Amt_UserID',
 'uid5',
 'uid5_TransactionDT_count',
 'uid5_TransactionDT_std',
 'uid_DT',
 'uid_DT_C10_mean',
 'uid_DT_C10_std',
 'uid_DT_C11_mean',
 'uid_DT_C11_std',
 'uid_DT_C12_mean',
 'uid_DT_C12_std',
 'uid_DT_C13_mean',
 'uid_DT_C13_std',
 'uid_DT_C14_mean',
 'uid_DT_C14_std',
 'uid_DT_C1_mean',
 'uid_DT_C1_std',
 'uid_DT_C2_mean',
 'uid_DT_C2_std',
 'uid_DT_C3_mean',
 'uid_DT_C3_std',
 'uid_DT_C4_mean',
 'uid_DT_C4_std',
 'uid_DT_C5_mean',
 'uid_DT_C5_std',
 'uid_DT_C6_mean',
 'uid_DT_C6_std',
 'uid_DT_C7_mean',
 'uid_DT_C7_std',
 'uid_DT_C8_mean',
 'uid_DT_C8_std',
 'uid_DT_C9_mean',
 'uid_DT_C9_std',
 'uid_DT_D10_mean',
 'uid_DT_D10_std',
 'uid_DT_D11_mean',
 'uid_DT_D11_std',
 'uid_DT_D12_mean',
 'uid_DT_D12_std',
 'uid_DT_D13_mean',
 'uid_DT_D13_std',
 'uid_DT_D14_mean',
 'uid_DT_D14_std',
 'uid_DT_D15_mean',
 'uid_DT_D15_std',
 'uid_DT_D2_mean',
 'uid_DT_D2_std',
 'uid_DT_D3_mean',
 'uid_DT_D3_std',
 'uid_DT_D4_mean',
 'uid_DT_D4_std',
 'uid_DT_D5_mean',
 'uid_DT_D5_std',
 'uid_DT_D6_mean',
 'uid_DT_D6_std',
 'uid_DT_D7_mean',
 'uid_DT_D7_std',
 'uid_DT_D8_mean',
 'uid_DT_D8_std',
 'uid_DT_D9_mean',
 'uid_DT_D9_std',
 'uid_DT_TransactionAmt_mean',
 'unique_Amt_UserID',
 'unique_Device',
 'unique_IP',
 'unique_Pemail_UserID',
 'unique_Remail_UserID',
 'unique_adr1_UserID',
 'unique_adr2_UserID',
 'unique_cards_userid']
# features = list of your features
#to_drop = [f for f in features_columns if '_DT_M_' in f or '_DT_W_' in f or '_DT_D_' in f or 'V_279_321_' in f]
#to_drop = ['uid5','groups']
#features_columns = list(set(features_columns) - set(to_drop))

print('Used Features:', len(features_columns))

#print('Dropped Features:', len(to_drop))

Used Features: 353


In [0]:
sorted(features_columns)

In [0]:
# XGB Params
xgb_params = {
              #'objective':'binary:logistic', #comment this if using focal loss
              'max_depth':9,
              'learning_rate':0.015,
              'subsample':0.9,
              'colsample_bytree':0.6,
              'tree_method':'hist',
              'eval_metric':'auc',
              'seed':SEED
              }

In [0]:
now = datetime.datetime.now() + datetime.timedelta(hours=2)
now = now.strftime("%H-%M_%d-%m-%Y")

In [0]:
@telegram_sender(token='', chat_id=0)
def xgboost_ieee_focal_all_feat():
  logfile = open("../content/gdrive/My Drive/IEEE fraud Kaggle 2019/xgb_output/log_f_xgb_focal_all_feat_"+str(now)+".txt","w")
  # Model Train
  if LOCAL_TEST:
    test_predictions, oof_score = make_predictions(train_df,
                                                   test_df,
                                                   features_columns,
                                                   TARGET,
                                                   xgb_params=xgb_params,
                                                   logfile=logfile,
                                                   custom_object = focal_binary_object)
  else:   
    start = time.time()
    test_predictions, oof_score = make_predictions(train_df,
                                                   test_df,
                                                   features_columns,
                                                   TARGET,
                                                   xgb_params=xgb_params,
                                                   logfile=logfile,
                                                   custom_object = focal_binary_object,
                                                   NFOLDS=5)
    print('-'*30)
    print('-'*30, file=logfile)
    print('Training time (min):', round((time.time() - start)/60,2))
    print('Training time (min):', round((time.time() - start)/60,2), file=logfile)
    print('Used Features:', len(features_columns), file=logfile)

  logfile.close()
  
  # Export
  if PREDICT:
      test_predictions['isFraud'] = test_predictions['prediction']
      test_predictions[['TransactionID','isFraud']].to_csv('../content/gdrive/My Drive/IEEE fraud Kaggle 2019/xgb_output/test_f_xgb_focal_all_feat_'+str(round(oof_score,4))+'.csv', index=False)

  return {"XGB - OOF AUC Score": round(oof_score,4)}

In [0]:
xgboost_ieee_focal_all_feat()

Fold: 0
Tr len: 453219 - Val len: 137321
[0]	train-auc:0.848969	val-auc:0.804572
[300]	train-auc:0.976382	val-auc:0.90222
[600]	train-auc:0.993421	val-auc:0.919648
[900]	train-auc:0.997354	val-auc:0.927014
[1200]	train-auc:0.99888	val-auc:0.930838
[1500]	train-auc:0.999597	val-auc:0.932438
[1799]	train-auc:0.999853	val-auc:0.933197
Train Fold AUC: 0.9998531287739796 - Val Fold AUC: 0.9331971897317478
Fold: 1
Tr len: 488908 - Val len: 101632
[0]	train-auc:0.837292	val-auc:0.832233
[300]	train-auc:0.974268	val-auc:0.933335
[600]	train-auc:0.992855	val-auc:0.949882
[900]	train-auc:0.996998	val-auc:0.953883
[1200]	train-auc:0.998781	val-auc:0.955199
[1500]	train-auc:0.999544	val-auc:0.956121
[1799]	train-auc:0.999831	val-auc:0.955864
Train Fold AUC: 0.9998309619204185 - Val Fold AUC: 0.9558642094585769
Fold: 2
Tr len: 497955 - Val len: 92585
[0]	train-auc:0.849603	val-auc:0.828886
[300]	train-auc:0.973658	val-auc:0.935881
[600]	train-auc:0.992414	val-auc:0.954167
[900]	train-auc:0.996766	v

{'XGB - OOF AUC Score': 0.9523}